# Run concentration driven for methane and investigate methane lifetime

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats

from tqdm import tqdm
from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, FAIR
from fair21.defaults import species_config_from_default

In [ ]:
df_temp = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = np.zeros(751)
gmst[100:271] = df_temp['gmst'].values
gmst[271:351] = np.linspace(gmst[270], 4, 80)
gmst[351:] = 4

In [ ]:
samples = 1000

In [ ]:
species_ids = {
    'ch4': SpeciesID('CH4', Category.CH4, run_mode=RunMode.CONCENTRATION),
    'co2': SpeciesID('CO2', Category.CO2, run_mode=RunMode.CONCENTRATION),
    'n2o': SpeciesID('N2O', Category.N2O, run_mode=RunMode.CONCENTRATION),
}

In [ ]:
concentration_species = ['CH4']
dummy_species = ['CO2', 'N2O']
species_to_include = concentration_species + dummy_species

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
emis_in = df_emis.loc[
    (df_emis['Scenario']=='ssp370') & (df_emis['Variable'].str.endswith("CH4")) & 
    (df_emis['Region']=='World'), '1750':'2500'
].interpolate(axis=1).values.squeeze()

In [ ]:
scenarios_to_include = ['ssp370']
scenarios = []

for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(concentration_species):
        species_rcmip_name = species.replace("-", "")
        conc_in = df_conc.loc[
            (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_conc['Region']=='World'), '1750':'2500'
        ].interpolate(axis=1).values.squeeze()

        list_of_species.append(Species(species_ids[species.lower()], concentration=conc_in))
    for species in dummy_species:
        list_of_species.append(Species(species_ids[species.lower()], concentration=np.ones(751)*species_config_from_default(species).baseline_concentration))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
ch4_lifetime = scipy.stats.uniform.rvs(7.25, 2, size=samples, random_state=31018316)
ch4_iirf_0 = np.ones(samples) * scipy.stats.uniform.rvs(7.25, 2, size=samples, random_state=10810610)
ch4_iirf_temperature = scipy.stats.uniform.rvs(-0.4, 0.35, size=samples, random_state=33372001)
ch4_iirf_airborne = scipy.stats.uniform.rvs(0.0001, 0.0005, size=samples, random_state=371210)
ch4_baseline_concentration = scipy.stats.norm.rvs(729.2, 9.4, size=samples, random_state=121270)

In [ ]:
configs = []

seedgen = 1876963
for imodel in tqdm(range(samples)):
    config_name = f"{imodel}"
    climate_response = ClimateResponse()
    seedgen = seedgen+399
    species_config = [species_config_from_default(species) for species in species_to_include]
    # CH4 is index 0
    #species_config[0].iirf_0 = ch4_iirf_0[imodel]
    species_config[0].iirf_airborne = ch4_iirf_airborne[imodel]
    species_config[0].iirf_temperature = ch4_iirf_temperature[imodel]
    species_config[0].lifetime = ch4_lifetime[imodel]
    species_config[0].baseline_concentration = ch4_baseline_concentration[imodel]
    species_config[0].natural_emissions_adjustment = 19.019783117809567
    species_config[0].calculate_iirf0()  # needed if editing the other params
    species_config[0].species_id = species_ids['ch4']
    species_config[1].species_id = species_ids['co2']
    species_config[2].species_id = species_ids['n2o']
    configs.append(Config(config_name, climate_response, species_config))

In [ ]:
configs[0].species_configs[0]

In [ ]:
start = time.time()
fair = FAIR(scenarios, configs, time=np.arange(1750.5, 2501))
fair.prescribe_temperature(gmst)
fair.run()
end = time.time()
print (f"{len(scenarios) * len(configs)} ensemble members in {end - start}s.")

In [ ]:
pl.plot(fair.concentration_array[:271,0,:,0,0]);
pl.plot(conc_in[:271], color='k')

In [ ]:
pl.plot(fair.emissions_array[:271,0,:,0,0]);
pl.plot(emis_in[:271], color='k')

In [ ]:
pl.hist(fair.emissions_array[264,0,:,0,0]);
pl.axvline(emis_in[264], color='k')

In [ ]:
pl.plot(fair.alpha_lifetime_array[:,0,:,0,0]);

In [ ]:
pl.plot(fair.alpha_lifetime_array[:,0,:,0,0] * fair.lifetime_array[:,0,:,0,0]);